# Using Bayesian optimization and multimodal data fusion to build a Algo Trading Framework
Outline of the steps to be followed to implement a trading strategy that uses Bayesian optimization and multimodal data fusion:

-    Collect and preprocess the data: Gather the relevant data sources, such as price data, news articles, social media posts, and economic indicators. Clean, normalize, and transform the data as needed to create a unified representation that captures the relevant information.

-    Define the objective function: Specify the performance metric that you want to optimize, such as the Sharpe ratio or the profit and loss (P&L). You can use the objective function to evaluate the performance of the trading strategy on a hold-out test set.

-    Define the machine learning model: Choose a suitable model architecture that can integrate multiple sources of data and capture the relevant patterns and trends. For example, you can use a CNN or a recurrent neural network (RNN) for text data, and a feedforward network or an LSTM network for numerical data. You can then fuse the outputs of the different models using a fusion layer that combines the representations into a joint feature space.

-    Define the hyperparameter search space: Specify the hyperparameters that you want to optimize, such as the learning rate, the number of hidden units, the regularization strength, and the dropout rate. You can use a probability distribution to sample the hyperparameters and update the probabilistic model.

-    Train and validate the model: Train the machine learning model on the training data and validate its performance on the validation set. Use the objective function (that maximizes the Sharpe ratio of the strategy) to evaluate the model's performance and update the hyperparameters using Bayesian optimization.

-    Test the model: Test the final optimized model on a hold-out test set to evaluate its performance in real-world conditions. Monitor its performance over time and adjust the strategy as needed.

In [ ]:
import pandas as pd
import numpy as np
import tweepy
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from bayes_opt import BayesianOptimization
from hyperopt import fmin, tpe, hp, Trials

### Collect and preprocess the data

In [ ]:
def preprocess_data(data):
    # Clean text data
    data['cleaned_text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
    data['cleaned_text'] = data['cleaned_text'].apply(lambda x: x.lower())
    data['cleaned_text'] = data['cleaned_text'].apply(lambda x: nltk.word_tokenize(x))
    stop_words = set(stopwords.words('english'))
    data['cleaned_text'] = data['cleaned_text'].apply(lambda x: [word for word in x if word not in stop_words])
    data['cleaned_text'] = data['cleaned_text'].apply(lambda x: ' '.join(x))
    
    # Scale numerical data
    scaler = MinMaxScaler()
    data['scaled_price'] = scaler.fit_transform(data['price'].values.reshape(-1, 1))
    
    # Calculate sentiment score
    sia = SentimentIntensityAnalyzer()
    data['sentiment_score'] = data['text'].apply(lambda x: sia.polarity_scores(x)['compound'])
    
    return data

### Collect the sentiment data from Twitter

In [ ]:
consumer_key = 'your_consumer_key'
consumer_secret = 'your_consumer_secret'
access_token = 'your_access_token'
access_token_secret = 'your_access_token_secret'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
search_term = 'S&P 500'
tweets = tweepy.Cursor(api.search_tweets, q=search_term, lang='en').items(1000)
tweet_df = pd.DataFrame(columns=['text'])
for tweet in tweets:
    tweet_df = tweet_df.append({'text': tweet.text}, ignore_index=True)

sentiment_data = preprocess_data(tweet_df)

### Create a unified representation of the data

In [ ]:
def fuse_data(text_data, numerical_data):
    # Merge text and numerical data
    fused_data = pd.concat([text_data[['cleaned_text', 'sentiment_score']], numerical_data[['scaled_price']]], axis=1)
    
    return fused_data

### Train a machine learning model

In [ ]:
def train_model(data, labels):
    # Define machine learning model architecture
    def build_model(learning_rate, dropout_rate, lstm_units, cnn_filters, cnn_kernel_size, fusion_units):
        # Define input layers
        text_input = tf.keras.layers.Input(shape=512, name='text_input')
        numerical_input = tf.keras.layers.Input(shape=(1,), name='numerical_input')

        # Define text input processing layers
        text_cnn = tf.keras.layers.Conv1D(filters=cnn_filters, kernel_size=cnn_kernel_size, activation='relu')(text_input)
        text_pool = tf.keras.layers.GlobalMaxPooling1D()(text_cnn)

        # Define numerical input processing layers
        numerical_lstm = tf.keras.layers.LSTM(units=lstm_units, dropout=dropout_rate)(numerical_input)

        # Define fusion layer
        fusion = tf.keras.layers.Concatenate()([text_pool, numerical_lstm])
        fusion = tf.keras.layers.Dense(units=fusion_units, activation='relu')(fusion)
        fusion = tf.keras.layers.Dropout(rate=dropout_rate)(fusion)

        # Define output layer
        output = tf.keras.layers.Dense(units=1, activation='sigmoid')(fusion)

        # Define machine learning model
        model = tf.keras.models.Model(inputs=[text_input, numerical_input], outputs=output)

        # Compile the model with Adam optimizer and binary crossentropy loss
        optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        return model

### Split the data into training and validation sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

### Define the hyperparameter search space

In [ ]:
parameter_space = {
    'learning_rate': hp.loguniform('learning_rate', -6, -2),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5),
    'lstm_units': hp.choice('lstm_units', [64, 128, 256]),
    'cnn_filters': hp.choice('cnn_filters', [64, 128, 256]),
    'cnn_kernel_size': hp.choice('cnn_kernel_size', [3, 5, 7]),
    'fusion_units': hp.choice('fusion_units', [64, 128, 256])
}

### Define the objective function

In [ ]:
@tf.function
def objective(hyperparameters):
    # Build the model with the hyperparameters
    model = build_model(hyperparameters['learning_rate'], hyperparameters['dropout_rate'], 
                        hyperparameters['lstm_units'], hyperparameters['cnn_filters'],
                        hyperparameters['cnn_kernel_size'], hyperparameters['fusion_units'])

    # Train the model on the training data
    history = model.fit([X_train_text, X_train_numerical], y_train, epochs=5, 
                        validation_data=([X_val_text, X_val_numerical], y_val),
                        verbose=0)

    # Evaluate the model on the validation data and compute the Sharpe ratio
    y_pred = model.predict([X_val_text, X_val_numerical]).flatten()
    returns = y_val * (y_pred - 0.5)
    sharpe_ratio = np.sqrt(252) * np.mean(returns) / np.std(returns)

    # Return the negative Sharpe ratio as the objective value to minimize
    return {'objective': -sharpe_ratio, 'status': 'ok'}

### Perform the hyperparameter search using Bayesian optimization

In [ ]:
trials = Trials()
best_hyperparameters = fmin(fn=objective, space=hyperparameter_space, algo=tpe.suggest, max_evals=100, trials=trials)

#Print the best hyperparameters found by Bayesian optimization
print('Best hyperparameters: ', best_hyperparameters)

### Train the final model with the best hyperparameters

In [ ]:
model = train_model(X_train, y_train, **best_hyperparameters)

### Evaluate the model on the test set

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

### Calculate predicted labels and probabilities on the test set

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = np.where(y_pred_prob > 0.5, 1, 0)

### Calculate Sharpe ratio on the test set

In [ ]:
test_returns = np.multiply(y_test, test_prices.pct_change().shift(-1))
test_strategy_returns = np.multiply(y_pred, test_returns)
test_sharpe_ratio = np.sqrt(252) * np.mean(test_strategy_returns) / np.std(test_strategy_returns)
print('Test Sharpe ratio: ', test_sharpe_ratio)


This is just a simple example, but the approach can be extended to more complex trading strategies and datasets. I hope you find this approach interesting and useful in your own trading endeavors. Let me know if you have any questions or feedback!

---

Feel free to modify code to your liking. I hope this helps!
